# SULO  Clinical Data Modeling Tutorial

This tutorial use the SULO ontology for representing typical healthcare concepts such as **Condition**, which refers to a person's health state as a *process* that the patient is experiencing; a **Procedure**, as an *activity or process* to assess, diagnose, treat or prevent a health condition; and **Measurement**, which is an *information entity* that represents the result of observing and/or quantifying some quality of a person, object or process. Measurements are always connected to sume qualitative and quantitative value (+ unit).

Along this tutorial we will represent an administrative case which describes a patient's encounter or stay at a healthcare provider institution. It can contain all clinical processes associated with the patient stay. E.g the admission and discharge events, observations, diagnoses, treatments or any assessment made to a patient. It may also include additional information such as the subject of care, or the location where the encounter or stay takes place.

Our administrative case will represent a care episode that includes the patient's admission to, and subsequent discharge from the healthcare facility. During the encounter, the patient's blood pressure is measured, and hypertension is diagnosed. The case will also document the dministration of medication and the issuance of a prescription for continued treatment following discharge.

First, we will initialize the environment as follows:

In [ ]:
!pip install owlready2 # Install dependencies
import os

def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

!rm -rf /content/*
!git clone https://github.com/CMCosta/SULO-health-data-modelling.git
%cd /content/SULO-health-data-modelling

!ls -lah lib
import sys
sys.path.insert(0, os.getcwd())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 34.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.49-py3-none-any.whl size=23742426 sha256=317315a0ea9f0625e985e074ae0e6d6e8eb755c4244e224cadc2f15196b9920e
  Stored in directory: /root/.cache/pip/wheels/43/fe/dc/a0de3c289cfd5923ece6524469d328950e14fa0c90b1088ffa
Successfully built owlready2
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u462-ga%7eus1-0ubuntu2%7e22.04.2_amd64.deb  404  Not Found [IP: 185.125.190.36 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u462-ga%7eus1-0ubuntu2%7e22.04.2_amd64.deb  404  Not Found [IP: 185.125.190.36 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
openjdk version "17.0.16" 2025-0

 Now, we will create our new ontology, which will import SULO. We will load the SULO ontology from it's web location and import it into our health ontology, so we can use SULO's classes and properties in our definitions.

In [ ]:
from owlready2 import *
from lib.helpers import *


# Load SULO ontology (assuming it's accessible at this URI)
sulo = get_ontology("https://w3id.org/sulo/sulo.owl").load()

# Define the new Healthcare-related extension ontology
health_ontology = get_ontology("https://w3id.org/sulo/extension/healthcare/")
health_ontology.imported_ontologies.append(sulo)

## 1. CREATION OF SOME OF THE MAIN CLINICAL ENTITY TYPES (as OWL classes)

We will create some of the main ontology classes needed for representing the administrative case mentioned above. For that, we need to classify these classes under the appropriate SULO top-level categories.

We will create classes for representing:

- **AdministrativeCase**: A process involving administrative and clinical events that occur during a care episode.
- **Admission**: A process by which a patient is accepted into a healthcare facility.
- **Discharge**: A process by which a patient is released from a healthcare facility.
- **Condition**: A process of health, disorder, or disease.
- **Measurement**: A quantitative or qualitative result of some measurement procedure.
- **UnitOfMeasurement**: A standardized quantity used to express and compare the magnitude of a physical quality.
- **MeasurementProcess**: A process used to determine the value of a specific physical, physiological, or clinical parameter in a patient.
- **PhysicalQuality**: Measurable or observable characteristic of a physical entity, such as its size, mass, temperature, pressure, or other physical property.
- **MedicationAdministration**: A process for representing the administration of medication.
- **MedicationPrescription**: A plan for administering medication, including the administration instructions.
- **PharmaceuticalProduct**: A clinical drug or medicinal product.
- **Patient**: A human who is the subject of care.
- **BodyPart**: A distinct anatomical structure or region of a living organism’s body.
- **SubjectOfCareRole**: A role in which an individual is the focus of healthcare activities.
- **ActiveRole**, **PassiveRole**:Roles in which the individual has an active or passive participation in a process respectively.

Next, we classify them under the right SULO top-level class:

In [ ]:
with health_ontology:
    class AdministrativeCase (sulo.Process):
        pass
    class Admission (sulo.Process):
        pass
    class Discharge (sulo.Process):
        pass
    class Condition (sulo.Process):
        pass
    class Measurement (sulo.Quantity):
        pass
    class UnitOfMeasurement (sulo.Unit):
        pass
    class MeasurementProcess (sulo.Process):
        pass
    class PhysicalQuality (sulo.Quality):
        pass
    class MedicationAdministration (sulo.Process):
        pass
    class MedicationPrescription (sulo.InformationObject):
        pass
    class PharmaceuticalProduct(sulo.SpatialObject):
        pass
    class HumanOrganism (sulo.SpatialObject):
        pass
    class Patient (HumanOrganism):
        pass
    class BodyPart (sulo.SpatialObject):
        pass
    class PassiveRole (sulo.Role):
        pass
    class ActiveRole (sulo.Role):
        pass
    class SubjectOfCareRole (PassiveRole):
        pass
    print("Main classes or entity types created")

Main classes or entity types created


### 2. PROCESSES, THEIR PARTICIPANTS, AND THE ROLES THEY PLAY - MEASUREMENT PROCESS

A process might involve many different objects, each playing a different *role*. To discriminate the participants we instantiate roles for each individual involved in the process. This is why SULO uses only the object property *hasParticipant* and no subproperties, according to its philosophy of keeping the set of object and datatype properties to a minimum.

In SULO we have defined the PRO (Process-Role-Object) Design pattern to provide a modular way to represent how (non-process) entities participate in processes through their specific roles. The pattern requires that particular roles are directly linked to a process instance using
hasParticipant and to their respective role holders using isFeatureOf. The pattern removes the need for
role-based relations such as hasInstrument, hasOutcome, hasPatient, etc.

In the running use case we want to represent a measurement process, the measurement of blood pressure. This is described as a **MeasurementProcess** that occurs or takes place at some **BodyPart**, has as result some **Measurement** and is measured using some **Device**.


As we can see below, both the quantity result of the measurement and the device used to measure, are both participants of the **MeasurementProcess**, with the roles **OutputRole** and **InstrumentRole** respectively.

In [ ]:
with health_ontology:
    class OutputRole (sulo.Role):
        pass
    class InstrumentRole (sulo.Role):
        pass
    class Device (sulo.SpatialObject):
        pass
    class MeasurementProcess (sulo.Process):
        is_a = [
            sulo.isIn.some(BodyPart), # body part where the bp is taken
            sulo.hasParticipant.some(OutputRole & sulo.isFeatureOf.some(Measurement)),
            sulo.hasParticipant.some(InstrumentRole & sulo.isFeatureOf.some(Device))
        ]
    print("Axioms added to MeasurementProcedure")

Axioms added to MeasurementProcedure


### 3. QUANTITATIVE PROCESS OUTPUTS - QUANTITATIVE OUTPUT OF A MEASUREMENT

Once we have represented the **MeasurementProcess** class, let's model its quantitative result with the class **Measurement**, which is an *information object*, more specifically a *quantity*. In our running use case we are measuring blood pressure, which is a *physical quality* (**PhysicalQuality**) of a *spatial object*, the arterial blood,  and its value has some **UnitOfMeasurement** as a reference. Here we use the SOLID (Single Object Literal Information Datum) Design Pattern, which uses the functional datatype property, *hasValue*, to assign a literal value to an instance of an *InformationObject*. This pattern transforms the introduction of domain-specific data properties such as *hasBloodPressure* by first extracting the implied classes (e.g., Blood pressure) that pertain to a relevant *InformationObject* (e.g. Measurement), and secondly finding the right relation to associate the information object to either a process or an object.

In [ ]:
with health_ontology:
   class Measurement (sulo.Quantity):
        is_a =[sulo.refersTo.some(PhysicalQuality),
               sulo.hasValue.some(float),
               sulo.hasPart.some(UnitOfMeasurement)
        ]
   print("Axioms added to Measurement")

Axioms added to Measurement


Next, we create specific classes for representing the blood pressure measurement process (**BloodPressureMeasurementProcess**) and the systolic blood pressure measurement (**SystolicBPMeasurement**) and diastolic blood pressure measurement (**DiastolicBPMeasurement**) results. We describe a **BloodPressureMeasurementProcess** as a **MeasurementProcess** that has as result both **SystolicBPMeasurement** and **DiastolicBPMeasurement**, which are both quantitative results.
In addition we create defined classes for **HighSystolicBPMeasurement** (Systolic >=150mmHg), **HighDiastolicBPMeasurement** (Diastolic >=90mmHg) and **HighBPMeasurement** as any or both of these two, in order to show some logic-based reasoning example.

In [ ]:
with health_ontology:
    class BloodPressure(PhysicalQuality):
        pass
    class SystolicBloodPressure (BloodPressure):
        pass
    class DiastolicBloodPressure (BloodPressure):
        pass
    class mmHG (UnitOfMeasurement):
        pass
    class BPMeasurement(Measurement):
         is_a = [sulo.refersTo.some(BloodPressure),
               sulo.hasValue.some(float),
               sulo.hasPart.some(UnitOfMeasurement)
        ]
    class SystolicBPMeasurement (BPMeasurement):
        is_a = [sulo.refersTo.some(SystolicBloodPressure),
               sulo.hasValue.some(float),
               sulo.hasPart.some(UnitOfMeasurement)
        ]
    class DiastolicBPMeasurement (BPMeasurement):
        is_a = [sulo.refersTo.some(DiastolicBloodPressure),
               sulo.hasValue.some(float),
               sulo.hasPart.some(UnitOfMeasurement)
        ]
    class BloodPressureMeasurementProcess (MeasurementProcess):
        is_a = [
            sulo.hasParticipant.some(OutputRole & sulo.isFeatureOf.some(SystolicBPMeasurement)),
            sulo.hasParticipant.some(OutputRole & sulo.isFeatureOf.some(DiastolicBPMeasurement))
        ]
    class HighSystolicBPMeasurement (SystolicBPMeasurement):
        equivalent_to  = [
               SystolicBPMeasurement &
               sulo.hasValue.some(ConstrainedDatatype(float, min_inclusive = 140))
               & sulo.hasPart.some(mmHG)
        ]
    class HighDiastolicBPMeasurement (DiastolicBPMeasurement):
        equivalent_to = [
               DiastolicBPMeasurement &
               sulo.hasValue.some(ConstrainedDatatype(float, min_inclusive = 90))
               & sulo.hasPart.some(mmHG)
        ]
    class HighBPMeasurement(BPMeasurement):
         equivalent_to = [
               HighSystolicBPMeasurement | HighDiastolicBPMeasurement
        ]
    print("BP measurement related classes created")

BP measurement related classes created


### 4. PROCESSES OCCUR IN SOME PLACE AND AT SOME TIME

We may want to register the (measured) time in which some process happened, whether in time instants, time intervals, or durations of time. In our running example this is for instance the case of the **Admission** and **Discharge** events. For both of them we have the instant of time (measurement) in which they occured. We will use the SULO property *atTime*. Note that all instances of time in SULO are results of time measurements, because the "absolute" time can only be approximated. In addition, we will represent where both processes happened (e.g. some healthcare facility) using the SULO property *isIn*.
Let's see how we can represent this using SULO.

In [ ]:
with health_ontology:
    class Admission (sulo.Process):
        is_a = [
            sulo.atTime.some(sulo.TimeInstant & sulo.hasValue.some(datetime.datetime)),
            sulo.isIn.some(sulo.SpatialObject)
        ]
    class Discharge (sulo.Process):
        is_a = [
            sulo.atTime.some(sulo.TimeInstant & sulo.hasValue.some(datetime.datetime)),
            sulo.isIn.some(sulo.SpatialObject)
        ]
    print("Temporal and location axioms added to Admission and Discharge")

Temporal and location axioms added to Admission and Discharge


### 5. HEALTHCARE CONDITIONS - DIAGNOSIS STATEMENT, CONDITION DIAGNOSED AND DIAGNOSTIC PROCESS

In our running example we want to represent an hypertension diagnosis. It is important to distinguish between the condition being diagnosed (**Condition**), which is a *process* in SULO, the diagnosis statement (**DiagnosisStatement**) wich is an *information object* and the diagnostic process (**DiagnosticProcess**) which is a *process*. In SULO we relate the condition (**Condition**) and the diagnosis statement (**DiagnosisStatement**) by using the relation *refersTo*, and this is a participant of the diagnostic process with the output role, since the statement is the result of the process.

In [ ]:
with health_ontology:
    class DiagnosisStatement(sulo.InformationObject):
        is_a = [
            sulo.refersTo.some(Condition)
        ]
    class DiagnosticProcess(sulo.Process):
        is_a = [
            sulo.hasParticipant.some(OutputRole & sulo.isFeatureOf.some(DiagnosisStatement))
        ]
    print("Diagnosis related classes created")

Next, we represent the specific **HypertensionCondition** as a defined class, indicating that there is hypertension if systolic or diastolic quantitative measurements are high (according to the values specified previously). We also provide defined classes for **HighSystolicBPCondition** and **HighDiastolicBPCondition**

In [ ]:
with health_ontology:
    class HypertensionCondition (Condition):
        equivalent_to = [
           Condition &
           sulo.hasParticipant.some(OutputRole and sulo.isFeatureOf.some(HighSystolicBPMeasurement)) |
           sulo.hasParticipant.some(OutputRole and sulo.isFeatureOf.some(HighDiastolicBPMeasurement))
       ]
    class HighSystolicBPCondition (Condition):
        equivalent_to = [
           Condition &
           sulo.hasParticipant.some(OutputRole and sulo.isFeatureOf.some(HighSystolicBPMeasurement))
       ]
    class HighDiastolicBPCondition (Condition):
        equivalent_to = [
           Condition &
           sulo.hasParticipant.some(OutputRole and sulo.isFeatureOf.some(HighDiastolicBPMeasurement))
       ]
    print("Hypertension and related classes created")

You can observe how if you have high systolic blood pressure or high diastolic blood pressure you have hypertension:

In [ ]:
in_ancestors(health_ontology, HighSystolicBPCondition, HypertensionCondition)

### 6. PROCESSES IN DIFFERENT STAGES, FROM PLANNING TO COMPLETION. MEDICATION ADMINISTRATION AND PRESCRIPTION

Clinical data refer to processes in different stages, from planning to completion. We will represent an already completed medication administration process as well as the prescription of some medication, in which the administration has not started yet.

To represent the administration of some medication, we first will create the **MedicationAdministration** class , which is a *process* in SULO. As described previously, in the process and their participants section, the specific medication or clinical drug (**PharmaceuticalProduct**) is one of the participants of the administration process. For the medication, we might need to indicate its dose (**PharmaceuticalDose**), which is a *quantity* in SULO as well as its dose form (**PharmaceuticalDoseForm**), which is a *quality* of the product.

In addition, we indicate the status of the administration process, **Completed**, as a subclass of **ProcessStatus**, which is a *quality* of the administration process in SULO. We can also indicate the time instant in which the medication was administered by using the SULO property *atTime*. The route of the administration by using *isIn* and indicating the body part (**BodyPart**) and the device **Device** used for administering it, if any. Last but not least, we can specify the patient and the person who administered the medication. All of them are participants of the administration process with their respective roles.

In [ ]:
with health_ontology:
    class SubstanceStrength(sulo.Quality):
        pass
    class PharmaceuticalDose(sulo.Quantity):
        sulo.refersTo.some(SubstanceStrength)
        sulo.hasValue.some(float),
        sulo.hasPart.some(UnitOfMeasurement)
    class PharmaceuticalDoseForm(sulo.Quality):
        pass
    class PharmaceuticalProduct(sulo.SpatialObject):
        is_a = [
            sulo.hasFeature.some(PharmaceuticalDose),
            sulo.hasFeature.some(PharmaceuticalDoseForm)
        ]
    class ProcessStatus(sulo.Quality):
        pass
    class Completed(ProcessStatus):
        pass
    class Patient(HumanOrganism):
        is_a = [
          sulo.hasFeature.some(SubjectOfCareRole)
        ]
    class MedicationAdministration(sulo.Process):
        is_a = [
            sulo.hasParticipant.some(PharmaceuticalProduct),
            sulo.hasFeature.some(ProcessStatus),
            sulo.atTime.some(sulo.TimeInstant & sulo.hasValue.some(datetime.datetime)),
            sulo.hasParticipant.some(InstrumentRole & sulo.isFeatureOf.some(Device)),
            sulo.hasParticipant.some(Patient),
            sulo.hasParticipant.some(ActiveRole & sulo.isFeatureOf.some(HumanOrganism)),
            sulo.isIn.some(BodyPart)
        ]
    print("Medication administration and related classes created")

The above representation indicates that the administration of certain medication was performed. However, we might want to describe that there is a prescription for such administration **MedicationPrescription** which is a treatment plan and thus an *information object* in SULO, but the administration has not been perfomed yet, which we will indicate as a feature or quality of the administration process which indicate its status as **NotStarted**. In the following we create the class for the **MedicationPrescription** and indicate that it refers to some **MedicationAdministration** process. Within the **MedicationAdministration** class, in this case the dose, dose form, etc. will indicate the instructions to actually administer the clinical drug. The reason of referreing to a collection (plurality) is that an individual administration process does not exist in this stage of prescription, and perhaps not even in the future (the patient refuses to buy or to ingest the drug). A direct reference from an OWL class to another OWL class is not available in OWL-DL (it would require so-called punning). As an approximation SULO recommends the reference to a collection, which can be seen as the set of the instances of an OWL class. A complete ontological analysis of that is still under discussion).     

In [ ]:
with health_ontology:
    class MedicationPrescription(sulo.InformationObject):
        sulo.refersTo.some(sulo.Collection & sulo.hasItem.some(MedicationAdministration))


### 7. PROCESSES AND PARTS

All the processes described above are part of our administrative case. In SULO, a process can have another processes as parts. We will indicate that **Admission**, **Discharge**, the **MeasurementProcess** and the **DiagnosticProcess** are all parts of the process **AdministrativeCase**. We will also state that it can only have one **Admission** and one **Discharge** processes maximum. Finally we will add the Patient as the subject of care to the case.

In [ ]:
with health_ontology:
    class AdministrativeCase(sulo.Process):
        is_a = [
             sulo.hasDirectPart.max(1,Admission),
             sulo.hasDirectPart.max(1,Discharge),
             sulo.hasDirectPart.some(BloodPressureMeasurementProcess),
             sulo.hasDirectPart.some(DiagnosticProcess),
             sulo.hasDirectPart.some(MedicationAdministration),
             sulo.hasParticipant.some(SubjectOfCareRole & sulo.isFeatureOf.some(HumanOrganism))
        ]
    print("Axioms added to AdministrativeCase .")

Finally, let's print the class tree:

In [ ]:

health_class_tree = get_color_tree([sulo, health_ontology])
print("Healthcare Class hierarchy")
display(health_class_tree)  # Class hierarchy

Finally, we save the ontology as follows:

In [ ]:
# Save the extended ontology as a Turtle file
health_ontology.save(file = "my-health-sulo.owl", format = "rdfxml")
print("Ontology saved to 'my-health-sulo.owl' in OWL/XML format.")

Issues not covered in this tutorial:

*   Realizable entities and the processes they are grounded in: similar problem of referring of non-instantiated classes as discussed in the case of medication prescription. Examples: allergies, risks, body functions
*   Factuality statements (known as verification status in FHIR). They are just parts of Diagnosis statements. Example: negated statements, suspected or differential diagnoses
.

